In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [9]:
def read_and_thresh(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    return thresh

In [10]:
def clean_image(thresh):
    # noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
    
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=3)
    
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    
    return sure_fg, unknown

In [11]:
def segment_watershed(sure_fg, unknown, img):
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    
    markers = cv2.watershed(img,markers)
    
    return markers

In [12]:
def cells_contours(markers):
    contours, hierarchy = cv2.findContours(markers,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    
    idx = 0
    cells = list()
    for c in contours:
        include = True  
        # omit this contour if it touches the edge of the image
        x,y,w,h = cv2.boundingRect(c)       
        if x <= 1 or y <=1:
            include = False                 
        if x+w+1 >= markers.shape[1] or y+h+1 >= markers.shape[0]:
            include = False
        # draw the contour
        if include == True and w>75 and h>75:
            cells.append(c)

    return cells

In [13]:
def pad_and_resize(im):
    desired_size = 500
    
    old_size = np.shape(im) # old_size is in (height, width) format
    
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    
    # new_size should be in (width, height) format
    
    im = cv2.resize(im, (new_size[1], new_size[0]))
    
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    
    return new_im

In [16]:
paths = ['20_B5_1_green.jpg', '20_B5_2_green.jpg', '21_B5_1_green.jpg', '21_B5_2_green.jpg', '22_B5_1_green.jpg','22_B5_2_green.jpg',
        '504_A4_1_green.jpg', '504_A4_3_green.jpg', '506_A4_2_green.jpg', '506_A4_3_green.jpg', '555_A4_1_green.jpg', '555_A4_2_green.jpg']

for path in paths:
    img = cv2.imread(path)
    
    thresh = read_and_thresh(img)
    sure_fg, unknown = clean_image(thresh)
    markers = segment_watershed(sure_fg, unknown, img)
    cells = cells_contours(markers)
        
    count = 0
    
    for i in range(1,len(cells),2):
        (x, y, w, h) = cv2.boundingRect(cells[i])
        new_img=img[y:y+h,x:x+w]
        resized_new_img = pad_and_resize(new_img)
        cv2.imwrite(path[:-4] + str(count) + ".jpg", resized_new_img)
        count = count+1
    
